In [1]:
#!/usr/bin python2.7

from astropy.io       import fits
from astropy.table    import Table
from astropy.time     import Time
from datetime         import date
from datetime         import datetime
import ephem
import fileinput
import getpass
import math
import matplotlib.pyplot                   as plt
import matplotlib                          as mpl
import numpy                               as np
from numpy            import linalg        as LA
import os
import pyfits
import scipy.optimize                      as optimization
from scipy            import integrate
from scipy            import interpolate
from scipy.optimize   import curve_fit
import subprocess
from subprocess       import Popen, PIPE
import urllib2
from work_module      import calculate
from work_module      import detector
from work_module      import readfile
from work_module      import writefile
calc = calculate()
det = detector()
rf = readfile()
wf = writefile()

In [4]:
fitsname = 'peak_eff_area_angle_calib_GBM_all.fits'
path = '/home/tkili/Work/calibration'
fitsfilepath = os.path.join(path, fitsname)
fitsfile = fits.open(fitsfilepath, mode='update')
data = fitsfile[1].data
fitsfile.close()
x = data.field(0)
y1 = data.field(1)#for NaI (33 keV)
y2 = data.field(2)#for NaI (279 keV)
y3 = data.field(3)#for NaI (662 keV)
y4 = data.field(4)#for BGO (898 keV)
y5 = data.field(5)#for BGO (1836 keV)

In [5]:
y1_fac = np.array([1., 1., 1., 1., 1., 0., 0., 0.])
y2_fac = np.array([0., 0., 5./279., 42./279., 165./279., 1., 0., 0.])
y3_fac = np.array([0., 0., 0., 0., 0., 138./662., 1., 1.])
                
#resulting effective area curve
y = []
for i in range(0, 8):
    p = y1_fac[i]*y1[90] + y2_fac[i]*y2[90] + y3_fac[i]*y3[90]
    y.append(p)
y = np.array(y)
print y

[ 113.50275243  113.50275243  114.43335207  121.31978947  144.21254081
   54.4455474    12.07951897   12.07951897]


array([-180., -178., -176., -174., -172., -170., -168., -166., -164.,
       -162., -160., -158., -156., -154., -152., -150., -148., -146.,
       -144., -142., -140., -138., -136., -134., -132., -130., -128.,
       -126., -124., -122., -120., -118., -116., -114., -112., -110.,
       -108., -106., -104., -102., -100.,  -98.,  -96.,  -94.,  -92.,
        -90.,  -88.,  -86.,  -84.,  -82.,  -80.,  -78.,  -76.,  -74.,
        -72.,  -70.,  -68.,  -66.,  -64.,  -62.,  -60.,  -58.,  -56.,
        -54.,  -52.,  -50.,  -48.,  -46.,  -44.,  -42.,  -40.,  -38.,
        -36.,  -34.,  -32.,  -30.,  -28.,  -26.,  -24.,  -22.,  -20.,
        -18.,  -16.,  -14.,  -12.,  -10.,   -8.,   -6.,   -4.,   -2.,
          0.,    2.,    4.,    6.,    8.,   10.,   12.,   14.,   16.,
         18.,   20.,   22.,   24.,   26.,   28.,   30.,   32.,   34.,
         36.,   38.,   40.,   42.,   44.,   46.,   48.,   50.,   52.,
         54.,   56.,   58.,   60.,   62.,   64.,   66.,   68.,   70.,
         72.,   74.,

In [7]:
data = readfile.earth_occ(readfile())
earth_ang_0 = data[0]
angle_d = data[1][0]
area_frac = data[2]
free_area = data[3][0]
occ_area = data[4]

In [12]:
muh = []
for echan in range(0, 8):
    y = y1_fac[echan]*y1 + y2_fac[echan]*y2 + y3_fac[echan]*y3
    
    #calculate the angle factors
    tck = interpolate.splrep(x, y)
    ang_fac = interpolate.splev(angle_d, tck, der=0)
    
    free_circ_eff = [free_area[0]*ang_fac[0]]
        
    for k in range(1, len(free_area)):
        circ_area = free_area[k] - free_area[k-1]
        circ_area_eff = circ_area*ang_fac[k]
        free_circ_eff.append(circ_area_eff)
        
    free_circ_eff = np.array(free_circ_eff)
        
    occ_circ_eff = []
        
    for j in range(0, len(earth_ang_0)):
        occ_circ_eff_0 = [occ_area[j][0]*ang_fac[0]]
        for i in range(1, len(occ_area[j])):
            circ_area = occ_area[j][i] - occ_area[j][i-1]
            circ_area_eff = circ_area*ang_fac[i]
            occ_circ_eff_0.append(circ_area_eff)
            
        occ_circ_eff.append(occ_circ_eff_0)
        
    occ_circ_eff = np.array(occ_circ_eff)

    eff_area_frac_0 = np.sum(occ_circ_eff, axis = 1)/np.sum(free_circ_eff)
    
    muh.append(eff_area_frac_0[0])

muh = np.array(muh)
muh = muh/muh[2]

print muh

[ 1.00151454  1.00151454  1.          0.98983815  0.96548746  0.87295057
  0.81295999  0.81295999]
